In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tabular-playground-series-jun-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jun-2021/train.csv
/kaggle/input/tabular-playground-series-jun-2021/test.csv


In [2]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from pprint import pprint
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from vecstack import stacking
from sklearn.metrics import accuracy_score

In [3]:
train_data = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/train.csv')
test_data = pd.read_csv('/kaggle/input/tabular-playground-series-jun-2021/test.csv')

In [4]:
def reduce_mem_usage(train_data):
    """ iterate through all the columns of a dataframe and modify the data type
    to reduce memory usage.
    """
    start_mem = train_data.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in train_data.columns:
        col_type = train_data[col].dtype

        if col_type != object:
            c_min = train_data[col].min()
            c_max = train_data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    train_data[col] = train_data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    train_data[col] = train_data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    train_data[col] = train_data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    train_data[col] = train_data[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    train_data[col] = train_data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    train_data[col] = train_data[col].astype(np.float32)
                else:
                    train_data[col] = train_data[col].astype(np.float64)
        else:
            train_data[col] = train_data[col].astype('category')

    end_mem = train_data.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))

    return train_data

In [5]:
train_data = reduce_mem_usage(train_data)
test_data = reduce_mem_usage(test_data)

Memory usage of dataframe is 117.49 MB
Memory usage after optimization is: 16.21 MB
Decreased by 86.2%
Memory usage of dataframe is 57.98 MB
Memory usage after optimization is: 8.01 MB
Decreased by 86.2%


In [6]:
train_data = train_data.drop('id',axis=1)
test_data = test_data.drop('id', axis=1)

y = train_data.target
X = train_data.drop('target',axis=1)



lencoder = LabelEncoder()
target = pd.DataFrame(lencoder.fit_transform(y))
target.head()

X_train,X_valid,y_train,y_valid = train_test_split(X, target, train_size=0.8, test_size=0.2,
                                                      random_state=0)

X_train = X_train.to_numpy()
y_train = y_train.to_numpy()
X_valid = X_valid.to_numpy()
y_valid = y_valid.to_numpy()

y_train = y_train.ravel()
y_valid = y_valid.ravel()

In [7]:
def log_loss(y_valid,pred_probs):
    pred_probs = pred_probs + 1e-15 # numerical stability
    y_valid = np.array(y_valid).reshape(-1, 1) 
    
    # get probs for each class
    preds_class = np.take_along_axis(pred_probs, y_valid, axis=1)
    return -1 * np.log(preds_class).mean()

In [8]:
xgb_params = {
    'colsample_bytree': 0,
    'gamma': 1.2274600449106712,
    'max_depth': 4,
    'min_child_weight': 4,
    'n_estimators': 593,
    'reg_alpha': 56,
    'reg_lambda': 0.39374361140115677,
    'random_state':0,
    'n_jobs':-1,
    'use_label_encoder':False,
    'objective':'multi:softprob'
    }

rf_params = {
    'bootstrap': True,
    'criterion': 'entropy',
    'max_depth': 77,
    'max_features': 'sqrt',
    'min_samples_leaf': 4,
    'min_samples_split': 2,
    'n_estimators': 500,
    'n_jobs':-1,
    'random_state':0
}

xgb_params2 = {
    'max_depth': 6,
    'subsample': 0.6486586332367784,
    'n_estimators': 3771,
    'lambda': 518.1848209502117,
    'learning_rate': 0.019382817636167057,
    'min_child_weight': 516,
    'colsample_bytree': 0.47077552288276003,
    'objective': 'multi:softprob', 
    'eval_metric': 'mlogloss', 
    'use_label_encoder': False,
    'num_class': 9, 
    'gpu_id': 0, 
    'tree_method': 'gpu_hist',
}

In [9]:
models = [
    RandomForestClassifier(**rf_params),
        
    XGBClassifier(**xgb_params2)
]

In [10]:
target = target.to_numpy().ravel()
X = X.to_numpy()

In [11]:
S_train, S_test = stacking(models,                   
                           X, target, test_data,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=True,
         
                           save_dir=None, 
            
                           metric=log_loss, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

task:         [classification]
n_classes:    [9]
metric:       [log_loss]
mode:         [oof_pred_bag]
n_models:     [2]

model  0:     [RandomForestClassifier]
    fold  0:  [1.75818816]
    fold  1:  [1.75464905]
    fold  2:  [1.75756251]
    fold  3:  [1.75853100]
    ----
    MEAN:     [1.75723268] + [0.00153154]
    FULL:     [1.75723268]

model  1:     [XGBClassifier]


/opt/conda/lib/python3.7/site-packages/xgboost/data.py:114: UserWarning: Use subset (sliced data) of np.ndarray is not recommended because it will generate extra copies and increase memory consumption
  "because it will generate extra copies and increase " +


    fold  0:  [1.74931042]
    fold  1:  [1.74439166]
    fold  2:  [1.74738010]
    fold  3:  [1.74954432]
    ----
    MEAN:     [1.74765663] + [0.00206367]
    FULL:     [1.74765663]



In [12]:
pd.DataFrame(S_train).head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.023463,0.020810,0.017611,0.005823,0.003816,0.674081,0.031785,0.165419,0.057191,0.018565,0.020236,0.017588,0.007054,0.004648,0.771300,0.025982,0.098163,0.036463
1,0.019115,0.124797,0.046471,0.010823,0.007626,0.502730,0.026488,0.121455,0.140495,0.023325,0.095139,0.059169,0.021600,0.007945,0.574606,0.025422,0.102074,0.090720
2,0.054795,0.108853,0.099402,0.026529,0.025475,0.194793,0.076151,0.258425,0.155577,0.040643,0.096283,0.076960,0.029662,0.024036,0.132146,0.083031,0.304678,0.212560
3,0.062579,0.090068,0.084731,0.027518,0.028926,0.171656,0.090584,0.299216,0.144722,0.056787,0.109069,0.099566,0.028155,0.031155,0.121789,0.076540,0.289481,0.187458
4,0.056274,0.328823,0.174410,0.017590,0.016763,0.123437,0.049274,0.076820,0.156610,0.055607,0.298823,0.158665,0.024645,0.021062,0.137112,0.040807,0.089423,0.173856


In [13]:
model = XGBClassifier(**xgb_params)
    
model = model.fit(S_train, target)
#y_pred = model.predict(S_test)

[17:22:05] WARNING: ../src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


## Testing with Catboost

In [14]:
cat_params ={
    'iterations': 10000, 
    'od_wait': 1115, 
    'learning_rate': 0.02248589308956038, 
    'reg_lambda': 86.12583478104304, 
    'subsample': 0.08594672381075155, 
    'random_strength': 29.926327447041192, 
    'depth': 6, 'min_data_in_leaf': 30, 
    'leaf_estimation_iterations': 3,
    'loss_function' : 'MultiClass',
    'eval_metric' : 'MultiClass',
    'bootstrap_type' : 'Bernoulli',
    'leaf_estimation_method' : 'Newton',
    'random_state' : 42,
    'task_type' : 'GPU',
    }

In [15]:
SX_train,SX_valid,Sy_train,Sy_valid = train_test_split(S_train, target, train_size=0.8, test_size=0.2,
                                                      random_state=0)

In [16]:
Sy_train.shape

(160000,)

In [17]:
from catboost import CatBoostClassifier
model2 = CatBoostClassifier(**cat_params)
model2 = model2.fit(SX_train, Sy_train,eval_set=[(SX_valid,Sy_valid)],early_stopping_rounds=100,verbose=True)

0:	learn: 2.1777260	test: 2.1776020	best: 2.1776020 (0)	total: 23.1ms	remaining: 3m 51s
1:	learn: 2.1580275	test: 2.1579074	best: 2.1579074 (1)	total: 37.7ms	remaining: 3m 8s
2:	learn: 2.1407715	test: 2.1405598	best: 2.1405598 (2)	total: 50.9ms	remaining: 2m 49s
3:	learn: 2.1230342	test: 2.1227711	best: 2.1227711 (3)	total: 65.1ms	remaining: 2m 42s
4:	learn: 2.1067186	test: 2.1064219	best: 2.1064219 (4)	total: 79.6ms	remaining: 2m 39s
5:	learn: 2.0950764	test: 2.0947500	best: 2.0947500 (5)	total: 86.1ms	remaining: 2m 23s
6:	learn: 2.0795922	test: 2.0791900	best: 2.0791900 (6)	total: 100ms	remaining: 2m 23s
7:	learn: 2.0650848	test: 2.0646297	best: 2.0646297 (7)	total: 115ms	remaining: 2m 23s
8:	learn: 2.0515486	test: 2.0510781	best: 2.0510781 (8)	total: 129ms	remaining: 2m 23s
9:	learn: 2.0380768	test: 2.0375566	best: 2.0375566 (9)	total: 143ms	remaining: 2m 23s
10:	learn: 2.0248840	test: 2.0242844	best: 2.0242844 (10)	total: 157ms	remaining: 2m 23s
11:	learn: 2.0127902	test: 2.0121645

In [18]:
y_pred = model2.predict_proba(S_test)

In [19]:
input_dir = "../input/tabular-playground-series-jun-2021/"
sub = pd.read_csv(input_dir + "sample_submission.csv")
sub.iloc[:, 1:] = y_pred
sub.to_csv("submission.csv", index=False)